In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from tickets import *
from minutes import *
from models import *

In [2]:
counts_2018 = np.genfromtxt('2018_count_250108_0209.csv', delimiter=',', skip_header=1)
day_min_counts = Minutes.day_min(counts_2018[:, 0])
print(f"Loaded {len(counts_2018)} rows of data")
print(f"The shape is {day_min_counts.shape}")

tickets_2018 = np.genfromtxt('2018_tickets_250108_0209.csv', delimiter=',', skip_header=1)
day_min_tickets = Minutes.day_min(tickets_2018)
print(f"Loaded {len(day_min_tickets)} rows of data")
print(f"The shape is {tickets_2018.shape}")
print()
print(f"The mean day is {np.mean(day_min_tickets[:, 0])}")
print(f"The std day is {np.std(day_min_tickets[:, 0])}")
print(f"The mean min is {np.mean(day_min_tickets[:, 1])}")
print(f"The std min is {np.std(day_min_tickets[:, 1])}")

Loaded 31856 rows of data
The shape is (31856, 2)
Loaded 160813 rows of data
The shape is (160813,)

The mean day is 29.77885494331926
The std day is 8.978221476042602
The mean min is 863.0213913054292
The std min is 255.18749935067723


In [5]:
all = Tickets()
all.add([[1, 60]])
day_min_all = Minutes.day_min(all.tickets)

parameters = [np.mean(day_min_tickets[:, 0]),
              np.std(day_min_tickets[:, 0]),
              np.mean(day_min_tickets[:, 1]),
              np.std(day_min_tickets[:, 1])]

transform = Minutes.get_transform(parameters)

day_c, min_c = transform(day_min_counts[:,0], day_min_counts[:,1])
radius = Minutes.radius(day_c, min_c)

day_a, min_a = transform(day_min_all[:,0], day_min_all[:,1])
radius_a = Minutes.radius(day_a, min_a)

print(f"Maximum radius is: {np.max(radius_a):.3f}")

Maximum radius is: 4.772


In [ ]:
def get_band_distribution(minute):



    evaluate_ticket_dist_i(intervals[0], intervals[1],
                             radius, counts_2018[:, 1],
                             radius_a)
    return d

In [ ]:
minute = Minutes.get_minutes([['April'], 25, 13, 10])
print(f"Minute: {minute[0]}")

d = get_band_distribution()

Minute: 35350
